In [1]:
from sqlalchemy import create_engine, text
from tabulate import tabulate

In [2]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [3]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [4]:
engine = create_engine(DATABASE_URL)

In [5]:
try:
    with engine.connect() as connection:
        
        connection.execute(text("DROP TABLE IF EXISTS Customers CASCADE;"))
        
        connection.execute(text("DROP TABLE IF EXISTS Orders CASCADE;"))
        
        connection.execute(text("""
            CREATE TABLE Customers (
                CustomerID SERIAL PRIMARY KEY,
                FirstName VARCHAR(255),
                LastName VARCHAR(255),
                Email VARCHAR(255) UNIQUE
            );
        """))

        connection.execute(text("""
            CREATE TABLE Orders (
                OrderID SERIAL PRIMARY KEY,
                CustomerID INT,
                OrderDate DATE,
                TotalAmount DECIMAL(10, 2),
                FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
            );
        """))

        connection.commit()
        
        print("Tables 'Customers' and 'Orders' created successfully.")
        
except Exception as e:
    print("Error creating tables:", e)

Tables 'Customers' and 'Orders' created successfully.


In [6]:
try:
    with engine.connect() as connection:
        
        connection.execute(
            text("""
            INSERT INTO Customers (FirstName, LastName, Email) VALUES
            ('John', 'Doe', 'john.doe@example.com'),
            ('Jane', 'Smith', 'jane.smith@example.com'),
            ('Peter', 'Jones', 'peter.jones@example.com');
        """)
        )

        connection.execute(
            text("""
            INSERT INTO Orders (CustomerID, OrderDate, TotalAmount) VALUES
            (1, '2023-11-01', 100.00),
            (1, '2023-11-15', 50.00),
            (2, '2023-11-05', 75.00),
            (3, '2023-11-10', 200.00),
            (3, '2023-11-20', 120.00);
        """)
        )

        connection.commit()
        
        print("Sample data inserted successfully.")
except Exception as e:
    print("Error inserting data:", e)

Sample data inserted successfully.


## SELECT: Retrieves data from one or more tables. 

## 1. Basic SELECT Syntax

In [7]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text("SELECT * FROM Customers"))
        
        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
            
except Exception as e:
    print("Error executing query:", e)

+--------------+-------------+------------+-------------------------+
|   customerid | firstname   | lastname   | email                   |
+==============+=============+============+=========================+
|            1 | John        | Doe        | john.doe@example.com    |
+--------------+-------------+------------+-------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com  |
+--------------+-------------+------------+-------------------------+
|            3 | Peter       | Jones      | peter.jones@example.com |
+--------------+-------------+------------+-------------------------+


## 2. Selecting Specific Columns

In [8]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text("SELECT FirstName, LastName FROM Customers"))
        
        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
except Exception as e:
    print("Error executing query:", e)

+-------------+------------+
| firstname   | lastname   |
+=============+============+
| John        | Doe        |
+-------------+------------+
| Jane        | Smith      |
+-------------+------------+
| Peter       | Jones      |
+-------------+------------+


## 3. WHERE Clause (Filtering):

In [9]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM Customers WHERE LastName = 'Doe'"))
        for row in result:
            print(row)
except Exception as e:
    print("Error executing query:", e)

(1, 'John', 'Doe', 'john.doe@example.com')


## 4. ORDER BY Clause (Sorting)

In [10]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM Customers ORDER BY LastName ASC"))
        
        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
except Exception as e:
    print("Error executing query:", e)

+--------------+-------------+------------+-------------------------+
|   customerid | firstname   | lastname   | email                   |
+==============+=============+============+=========================+
|            1 | John        | Doe        | john.doe@example.com    |
+--------------+-------------+------------+-------------------------+
|            3 | Peter       | Jones      | peter.jones@example.com |
+--------------+-------------+------------+-------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com  |
+--------------+-------------+------------+-------------------------+


## 5. LIMIT and OFFSET (Pagination)

In [11]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT * FROM Customers LIMIT 2 OFFSET 1"))
        for row in result:
            print(row)
except Exception as e:
    print("Error executing query:", e)

(2, 'Jane', 'Smith', 'jane.smith@example.com')
(3, 'Peter', 'Jones', 'peter.jones@example.com')


## 6. Aggregate Functions (COUNT, SUM, AVG, etc.)

In [12]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT COUNT(*) FROM Orders"))
        count = result.scalar()
        print(f"Total number of orders: {count}")

        result = connection.execute(text("SELECT SUM(TotalAmount) FROM Orders"))
        total_amount = result.scalar()
        print(f"Total amount of all orders: {total_amount}")

        result = connection.execute(text("SELECT AVG(TotalAmount) FROM Orders"))
        avg_amount = result.scalar()
        print(f"Average order amount: {avg_amount}")

except Exception as e:
    print("Error executing query:", e)

Total number of orders: 5
Total amount of all orders: 545.00
Average order amount: 109.0000000000000000


## 7. GROUP BY Clause

In [13]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("""
            SELECT CustomerID, COUNT(*) AS OrderCount
            FROM Orders
            GROUP BY CustomerID
        """))

        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
except Exception as e:
    print("Error executing query:", e)

+--------------+--------------+
|   customerid |   ordercount |
+==============+==============+
|            3 |            2 |
+--------------+--------------+
|            2 |            1 |
+--------------+--------------+
|            1 |            2 |
+--------------+--------------+


## 8. Subquery

In [14]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("""
            SELECT *
            FROM Customers
            WHERE CustomerID IN (SELECT CustomerID FROM Orders WHERE TotalAmount > 100)
        """))

        for row in result:
            print(row)
except Exception as e:
    print("Error executing query:", e)

(3, 'Peter', 'Jones', 'peter.jones@example.com')


In [15]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text("SELECT * FROM Customers"))
        
        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
            
except Exception as e:
    print("Error executing query:", e)

+--------------+-------------+------------+-------------------------+
|   customerid | firstname   | lastname   | email                   |
+==============+=============+============+=========================+
|            1 | John        | Doe        | john.doe@example.com    |
+--------------+-------------+------------+-------------------------+
|            2 | Jane        | Smith      | jane.smith@example.com  |
+--------------+-------------+------------+-------------------------+
|            3 | Peter       | Jones      | peter.jones@example.com |
+--------------+-------------+------------+-------------------------+


In [16]:
try:
    with engine.connect() as connection:
        
        result = connection.execute(text("SELECT * FROM Orders"))
        
        rows = result.fetchall()

        column_names = result.keys()

        table = tabulate(rows, headers=column_names, tablefmt="grid")

        print(table)
            
except Exception as e:
    print("Error executing query:", e)

+-----------+--------------+-------------+---------------+
|   orderid |   customerid | orderdate   |   totalamount |
+===========+==============+=============+===============+
|         1 |            1 | 2023-11-01  |           100 |
+-----------+--------------+-------------+---------------+
|         2 |            1 | 2023-11-15  |            50 |
+-----------+--------------+-------------+---------------+
|         3 |            2 | 2023-11-05  |            75 |
+-----------+--------------+-------------+---------------+
|         4 |            3 | 2023-11-10  |           200 |
+-----------+--------------+-------------+---------------+
|         5 |            3 | 2023-11-20  |           120 |
+-----------+--------------+-------------+---------------+


## INSERT (Adding a New Customer):

In [17]:
try:
    with engine.connect() as connection:
        connection.execute(
            text("""
            INSERT INTO Customers (FirstName, LastName, Email)
            VALUES (:firstname, :lastname, :email)
        """),
            {"firstname": "Alice", "lastname": "Brown", "email": "alice.brown@example.com"}
        )

        connection.commit()
        print("New customer inserted successfully.")
except Exception as e:
    print("Error inserting customer:", e)

New customer inserted successfully.


## UPDATE (Modifying Customer Information)

In [18]:
try:
    with engine.connect() as connection:
        connection.execute(
            text("""
            UPDATE Customers
            SET Email = :new_email
            WHERE FirstName = :firstname AND LastName = :lastname
        """),
            {"new_email": "alice.b@example.com", "firstname": "Alice", "lastname": "Brown"}
        )

        connection.commit()
        print("Customer email updated successfully.")
except Exception as e:
    print("Error updating customer:", e)

Customer email updated successfully.


## DELETE (Removing a Customer)

In [20]:
try:
    with engine.connect() as connection:
        
        connection.execute(
            text("""
            DELETE FROM Customers
            WHERE FirstName = :firstname AND LastName = :lastname
        """),
            {"firstname": "Alice", "lastname": "Brown"}
        )

        connection.commit()
        print("Customer deleted successfully.")
except Exception as e:
    print("Error deleting customer:", e)

Customer deleted successfully.
